In [3]:
import pandas as pd
import interface as itf

In [4]:
df = pd.read_csv('../dataset/clean/day_data_fixed.csv', header=0)

In [7]:
all_dfs = []
for pid in range(1, 17, 1):
    ex_df = itf.get_exercise(pid)
    ex_df['dateTime'] = pd.to_datetime(pd.to_datetime(ex_df['startTime']).dt.date)
    ex_df = ex_df.drop(columns=['logId', 'activityTypeId', 'activityLevel', 'startTime'])
    
    agg_df = ex_df.groupby('dateTime').agg(
        {
            'activityName': set,
            'averageHeartRate': 'mean',
            'calories': 'sum',
            'activeDuration': 'sum',
            'steps': 'sum',
            'elevationGain': 'sum'
        }
    ).reset_index()
    agg_df['activeDuration'] /= (1000*60)
    agg_df = agg_df.assign(pid=pid)
    agg_df = agg_df.rename(columns={'steps': 'active_steps', 'averageHeartRate':'active_average_hr', 'calories': 'active_calories'})
    
    all_dfs.append(agg_df)
    print(pid)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [8]:
total_ex_df = pd.concat(all_dfs, axis=0, ignore_index=True)

In [9]:
print(total_ex_df)

       dateTime    activityName  active_average_hr  active_calories  \
0    2019-11-01          {Walk}               94.0              494   
1    2019-11-02          {Walk}               98.0              354   
2    2019-11-04          {Walk}               97.0              145   
3    2019-11-05     {Walk, Run}              118.5              517   
4    2019-11-08          {Walk}               97.0              252   
...         ...             ...                ...              ...   
1367 2020-02-22          {Walk}              110.5              372   
1368 2020-03-09          {Walk}              102.0               92   
1369 2020-03-16          {Walk}              106.0               95   
1370 2020-03-20          {Walk}              106.0              109   
1371 2020-03-29  {Outdoor Bike}               71.0               75   

      activeDuration  active_steps  elevationGain  pid  
0          58.883333        4664.0         51.816    1  
1          40.966667        3035.

In [15]:
df['dateTime'] = pd.to_datetime(df['dateTime'])

In [16]:
final_df = df.merge(total_ex_df, on=['dateTime', 'pid'], how='left')

In [18]:
final_df.to_csv('../dataset/clean/day_data_v3.csv', index=False)